### Trainieren von LSTM auf den Aktienkurs

In [ ]:
!pip install bokeh
!pip install tensorflow
!pip install selenium
!pip install pandas_datareader
#!conda update -n base conda
#!conda install -c conda-forge -y firefox geckodriver


In [ ]:
REF_TIME = 0  # Referenzzeitpunkt
EPOCHS = 5
TEST_SIZE = 0.3
TIME_AHEAD = 5 #prediction step
BATCH_SIZE = 1
UNITS = 25

### utils

In [ ]:
import subprocess
import os
import sys
import glob
import json
import numpy as np
import time
from ast import literal_eval
import matplotlib.pyplot as plt
from bokeh.plotting import figure
from bokeh.io import show, export_svgs
from IPython.display import SVG, display


def get_number_processors():
    """Liefert die Anzahl der Prozessoren in einer CPU."""
    try:
        num = os.cpu_count()
    except Exception:
        # force exception in case mutiprocessing is not installed
        import multiprocessing
        num = multiprocessing.cpu_count()
    return num


def to_1dimension(df, step_size):
    """Umwandlung in das LSTM-Zeitreihenformat"""
    X, y = [], []
    for i in range(len(df)-step_size-1):
        data = df[i:(i+step_size), 0]
        X.append(data)
#        y.append(df[i + step_size, 0])
        y.append(df[REF_TIME, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    return X, y


def plot_series(values, xlabel=None, ylabel=None, color='b', legend=None):
    """Allgemeine Funktion zum Plotten von Serien"""
    xx = np.arange(1, len(values) + 1, 1)
    plt.plot(xx, values, color, label=legend)
    plt.legend(loc='upper left')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


def plot_series_prediction(true_values, train_predict, test_predict,
                           time_ahead=1, title=None, xlabel=None, ylabel=None,
                           color=['green', 'red', 'blue'],
                           legend=[None, None, None]):
    """Plotten Sie wahre Werte, Zugvorhersage und Testvorhersage mit Bokeh."""
    TOOLS = 'pan,wheel_zoom,box_zoom,reset,save,box_select'
    # x axis
    xx = np.array(range(true_values.shape[0]))
    xx1 = np.array(range(time_ahead, len(train_predict)+time_ahead))
    xx2 = np.array(
        range(len(train_predict)+(time_ahead*2)+1, len(true_values)-1))

    # figure
    p = figure(title=title, tools=TOOLS)
    p.line(xx, true_values.squeeze(),
           legend=legend[0], line_color=color[0], line_width=2)
    p.line(xx1, train_predict.squeeze(),
           legend=legend[1], line_color=color[1], line_width=1)
    p.line(xx2, test_predict.squeeze(),
           legend=legend[2], line_color=color[2], line_width=1)
    p.axis[0].axis_label = xlabel
    p.axis[1].axis_label = ylabel
    p.legend.location = "top_left"
    show(p)

    # save to svg so it is rendered on github
    p.output_backend = "svg"
    export_svgs(p, filename=title + '_.svg')
    display(SVG(title + '_.svg'))


In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import selenium
from bokeh.io import output_notebook
#from utils import (get_number_processors, plot_series, to_1dimension, plot_series_prediction)

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Tensorflow: ", tf.__version__)
print("Numpy: ", np.__version__)
print("Selenium: ", selenium.__version__)
print("Number of CPU processors: ", get_number_processors())

%load_ext autoreload
%autoreload 2

output_notebook()

### Daten

Die Daten, die wir verwenden werden, sind die Siemens AG-Aktienkurse von 2015 bis 2020. Der Zugriff auf die Daten ist direkt in[Yahoo finance](https://finance.yahoo.com/quote/SIEGY/history?p=SIEGY) möglich. Alternativ stellen wir Daten von Bitcoin bis USD zur Verfügung, die im selben Web zu finden sind.

Der Parameter `TIME_AHEAD` legt die Anzahl der Tage fest, die Sie im Voraus vorhersagen möchten. Je mehr Sie in der Zukunft vorhersagen wollen, desto ungenauer wird die Vorhersage sein.

In [ ]:
from pandas_datareader import data as web
import datetime as dt

# Zeitraum festlegen
start = dt.datetime(2015,1,1)
ende = dt.datetime(2020,8,14)

# SIEGY (Siemens AG) Daten von Yahoo Finance laden
df = web.DataReader('SIEGY', 'yahoo', start, ende)

# Daten ausgeben
print(df.shape)
print(df.head())

df = df.drop(['Adj Close', 'Volume'], axis=1)
print(df.shape)
print(df.head())

In [ ]:
#DATA = os.path.join('.', 'SIEGY_2015_2020.csv')
DATA = os.path.join('.', 'MSFT_2015_2020.csv')
#DATA = os.path.join('data', 'MSFT_2015_2020.csv')
#DATA = os.path.join('data', 'MSFT_2012_2017.csv')
#DATA = os.path.join('data', 'BTC-USD_2012_2017.csv')
df = pd.read_csv(DATA)

# Daten ausgeben
print(df.shape)
print(df.head())

df = df.drop(['Adj Close', 'Volume'], axis=1)
print(df.shape)
print(df.head())

Um das Problem zu vereinfachen, werden wir den Durchschnitt der Werte Open, High, Low und Close als Input nehmen.

In [ ]:
mean_price = df.mean(axis = 1)
plot_series(mean_price, xlabel='Days', ylabel='Mean value der Siemens AG. Aktie', color='b', legend='Mean price')

Der nächste Schritt ist die Skalierung des Datensatzes, was im Allgemeinen dazu beiträgt, dass die Vorhersage genauer ist.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)) #other typical scale values are -1,1
mean_price = scaler.fit_transform(np.reshape(mean_price.values, (len(mean_price),1)))

Sobald die Daten skaliert sind, können wir den Zug und die Testsets erstellen.

In [ ]:
train, test = train_test_split(mean_price, test_size=TEST_SIZE, shuffle=False)
print(train.shape)
print(test.shape)

In [ ]:
X_train, y_train = to_1dimension(train, TIME_AHEAD)
X_test, y_test = to_1dimension(test, TIME_AHEAD)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### LSTM

Lassen Sie uns das LSTM-Modell trainieren. Zur Optimierung verwenden wir den ADAM-Algorithmus. In Zeitreihen neigen adaptive Optimierungsmethoden dazu, bessere Ergebnisse zu erzielen als herkömmliche stochastische Gradientenabstiegsverfahren.

In [ ]:
def create_symbol(X, units=10, activation='linear', time_ahead=1):
    cell = tf.nn.rnn_cell.LSTMCell(units)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    sym = tf.layers.dense(outputs[-1], 1, activation=None, name='output')#activation=None (linear activation)
    return sym

In [ ]:
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 1, TIME_AHEAD])
y = tf.placeholder(tf.float32, shape=[None])

sym = create_symbol(X, units=UNITS, time_ahead=TIME_AHEAD)

loss = tf.reduce_mean(tf.squared_difference(sym, y)) #mse
optimizer = tf.train.AdamOptimizer()
model = optimizer.minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
for i in range(EPOCHS):
    ii = 0
    while(ii + BATCH_SIZE) <= len(X_train):
        X_batch = X_train[ii:ii+BATCH_SIZE,:,:]
        y_batch = y_train[ii:ii+BATCH_SIZE]
        sess.run(model, feed_dict={X:X_batch, y:y_batch})
        ii += BATCH_SIZE
    loss_train = sess.run(loss, feed_dict={X: X_batch, y: y_batch})
    print('Epoch {}/{}'.format(i+1, EPOCHS), ' Current loss: {}'.format(loss_train))

Sobald das Modell trainiert ist, können wir auf dem Testset vorhersagen und den RMSE-Wert erhalten. Die Modellvorhersage `pred_test` hat die Form von `(num_rows, TIME_AHEAD)`, mit den vorhergesagten Werten für die nächsten `TIME_AHEAD` Tage.

In [ ]:
y_guess = np.zeros(y_test.shape[0], dtype=np.float32)
ii = 0
while(ii + BATCH_SIZE) <= len(X_test):
    X_batch = X_test[ii:ii+BATCH_SIZE,:,:]
    output = sess.run(sym, feed_dict={X:X_batch})
    y_guess[ii:ii+BATCH_SIZE] = output
    ii += BATCH_SIZE

In [ ]:
y_test_inv = scaler.inverse_transform([y_test])
pred_test = scaler.inverse_transform([y_guess])
score = math.sqrt(mean_squared_error(y_test_inv, pred_test))
print('Test RMSE: %.2f' % (score))

In [ ]:
mean_price_inv = scaler.inverse_transform(mean_price)
train_set = mean_price_inv[:X_train.shape[0]]
plot_series_prediction(mean_price_inv, train_set, pred_test.T, time_ahead=TIME_AHEAD,
                       title='Siemens AG Vorhersage fuer ' + str(TIME_AHEAD) + ' Tage', 
                       xlabel='Days', ylabel='Mean value der Siemens AG. Aktie', 
                       legend=['True value','Training set','Test prediction'])

### Fazit
In diesem Tutorial haben wir gesehen, wie ein LSTM verwendet werden kann, um den Aktienkurs bei einer zeitlichen Reihe von Werten vorherzusagen.